### 필요한 라이브러리들만 import

In [2]:
import os, glob
import logging

from pprint import pprint
import json
import nltk
import numpy as np
import tensorflow as tf
from tensorflow import keras

# 에러 메세지만 로깅한다
tf.get_logger().setLevel(logging.ERROR)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 몇몇 필요 변수들을 미리 지정한다

In [4]:
# The percentage of the dataset you want to split as train and test
TRAIN_TEST_SPLIT = 0.1

MAX_INPUT_LENGTH = 512  # Maximum length of the input to the Encoder
MIN_TARGET_LENGTH = 5  # Minimum length of the output by Decoder
MAX_TARGET_LENGTH = 128  # Maximum length of the output by Decoder
BATCH_SIZE = 8  # Batch-size for training our model
LEARNING_RATE = 2e-5  # Learning-rate for training our model
MAX_EPOCHS = 1  # Maximum number of epochs we will train the model for

# HF model hub에서 가져올 모델 이름
MODEL_CHECKPOINT = "psyche/KoT5-summarization"

# Dataset 로딩 하기

문서요약 텍스트 샘플이라는 간단한 한글 말뭉치 데이터를 다운받는다.

그 후 ,다운 받은 데이터셋을 Hugging Face Datasets 포맷으로 변환한다.

이미 Hugging Face hub에 올라온 데이터셋의 경우는 load_dataset 함수를 통해 바로 사용이 가능하다.

In [5]:
from datasets import Dataset, load_dataset
ds_dir1 = './drive/MyDrive/Datasets/Texts/09.literature/20per/'
ds_dir2 = './drive/MyDrive/Datasets/Texts/09.literature/2~3sent/'
ds_path = glob.glob(ds_dir1 + '*.json') + glob.glob(ds_dir2 + '*.json')
len(ds_path)

9600

In [ ]:
doc_id, document, summary = [], [], []

for path_ in ds_path:

    with open(path_, 'r') as f:

        doc = json.load(f)

        doc_id.append(doc["Meta(Acqusition)"]["doc_id"])
        document.append(doc['Meta(Refine)']['passage'])
        summary.append(doc['Annotation']['summary1'])

raw_datasets = Dataset.from_dict({'id': doc_id,
                                  'document': document,
                                  'summary': summary})

In [ ]:
pprint(raw_datasets[1])

In [8]:
pprint(raw_datasets)

Dataset({
    features: ['id', 'document', 'summary'],
    num_rows: 4800
})


In [9]:
raw_datasets = raw_datasets.train_test_split(test_size=TRAIN_TEST_SPLIT)


# 데이터 전처리

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

Downloading:   0%|          | 0.00/2.46k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

In [11]:
if MODEL_CHECKPOINT in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b", "psyche/KoT5-summarization"]:
    prefix = "요약: "
else:
    prefix = ""

### 전처리 함수 선언하기
- prefix 추가 및 tokenization
- token_type_ids, attention_mask 생성

In [12]:
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"], max_length=MAX_TARGET_LENGTH, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

dataset 오브젝트의 map method를 통해 전처리 함수를 적용한다.
- dataset 내에 존재하는 train, valid, test 데이터셋에 한번에 적용 가능하다.

In [13]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/480 [00:00<?, ? examples/s]

Map:   0%|          | 0/480 [00:00<?, ? examples/s]

In [14]:
pprint(tokenized_datasets['train'][0])

{'attention_mask': [1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
           

# 모델 정의

In [15]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
# 요약 task는 seq2seq task로 분류된다

model = TFAutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT, from_pt=True)
# 원본이 pytorch 이므로 from_pt=True

Downloading:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5ForConditionalGeneration: ['encoder.embed_tokens.weight', 'lm_head.weight', 'decoder.embed_tokens.weight']
- This IS expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


S2S 모델을 학습하기 위해서는 적합한 data collator가 필요하다.
- input 뿐만 아니라 labels에 대해서도 padding 처리가 필요하기 때문이다
- DataCollatorForSeq2Seq를 사용하면 편리하게 가능하다.
- Keras를 이용하고 있으므로 tf.Tensor로 반환 받을 수 있도록 return_tensors='tf'로 설정한다.

In [16]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors='tf')

## 각 dataset 생성시, data_collator 적용시키기
- 학습시에 계산할 ROUGE score를 위해 generation_dataset을 추가로 생성한다.

In [17]:
train_dataset = tokenized_datasets["train"].to_tf_dataset(
    batch_size=BATCH_SIZE,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=True,
    collate_fn=data_collator,
)
test_dataset = tokenized_datasets["test"].to_tf_dataset(
    batch_size=BATCH_SIZE,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    collate_fn=data_collator,
)
generation_dataset = (
    tokenized_datasets["test"]
    .shuffle()
    .select(list(range(200)))
    .to_tf_dataset(
        batch_size=BATCH_SIZE,
        columns=["input_ids", "attention_mask", "labels"],
        shuffle=False,
        collate_fn=data_collator,
    )
)

# 모델링 및 컴파일링

In [18]:
optimizer = keras.optimizers.AdamW(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer) # loss 값은 내부적으로 처리하도록 설정

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


# 모델 학습 및 평가

학습을 진행하는 동안 동시에 모델을 평가하기 위해 metric_fn을 정의해서 ground_truth와 prediction간의 ROUGE score를 계산

In [19]:
import keras_nlp

rouge_l = keras_nlp.metrics.RougeL()


def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge_l(decoded_labels, decoded_predictions)
    # We will print only the F1 score, you can use other aggregation metrics as well
    result = {"RougeL": result["f1_score"]}

    return result

In [24]:
first_batch = next(iter(train_dataset.batch(1)))
print(first_batch['input_ids'].shape)

(1, 8, 368)


In [25]:
first_batch

{'input_ids': <tf.Tensor: shape=(1, 8, 368), dtype=int64, numpy=
 array([[[16038, 26202,   142, ...,     0,     0,     0],
         [16038, 26202,   334, ...,     0,     0,     0],
         [16038, 26202,   438, ...,     0,     0,     0],
         ...,
         [16038, 26202, 14939, ..., 25892, 25889,     1],
         [16038, 26202, 25889, ...,     0,     0,     0],
         [16038, 26202,  1312, ...,     0,     0,     0]]])>,
 'attention_mask': <tf.Tensor: shape=(1, 8, 368), dtype=int64, numpy=
 array([[[1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         ...,
         [1, 1, 1, ..., 1, 1, 1],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0]]])>,
 'labels': <tf.Tensor: shape=(1, 8, 39), dtype=int64, numpy=
 array([[[    3, 25983, 24063,  6158, 26772, 26042, 16098, 26057,  1856,
          18270, 22213,  4992, 26003,  1304,    39, 26346, 25894,   123,
          25906,   764,   555, 12764, 16250,  9758,   678,  2527, 2588

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(
    metric_fn, eval_dataset=generation_dataset, predict_with_generate=True
)

callbacks = [metric_callback]

# For now we will use our test set as our validation_data
model.fit(
    train_dataset, validation_data=test_dataset, epochs=MAX_EPOCHS, callbacks=callbacks
)

60/60 [==============================] - 258s 3s/step - loss: 1.9531 - val_loss: 1.8093 - RougeL: 0.0325


# Inference

Huggingface의 pipeline 메소드에서 제공하는 summarization을 사용하면 쉽게 구현이 가능하다.

- pipeline 메소드에 학습한 모델과 토크나이저를 인자로 넣어준다.
- 모델이 TF로 학습되었으므로 framework='tf'도 함께 인자로 넣어준다.

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, framework="tf")

summary = summarizer(
            raw_datasets["test"][0]["document"],
            min_length=MIN_TARGET_LENGTH,
            max_length=MAX_TARGET_LENGTH,
        )

pprint(f'document :{raw_datasets["test"][0]["document"]}')
pprint(f'label summary :{raw_datasets["test"][0]["summary"]}')
pprint(f'pred summary :{summary[0]["summary_text"]}')

('document :마스카니의 제자 점심때쯤 해서 작곡가 마스카니는 정신을 집중시키기 위하여 책상 앞에 앉 은 후 눈을 감았습니다 . 저 '
 '음에서 고음으로, 고음에서 저음으로 아름다운 멜로디는 그의 머릿속에 떠올랐습니다. 마침 그때 불의에 일어난 창 밖의 요란한 소리는 그의 '
 '환상의 실마리를 끊 어놓았습니다. 길거리 창 밑에는 어떤 걸인 풍금수(風琴手)가 제멋에 겨워 서 한참 신이 나가지고는 되는 대로 풍금을 '
 '타고 있었습니다. 여기에 화가 난 마스카니는 들었던 펜을 내던지고 두 주먹을 불끈 쥐고 일어나려다가 다 시 가만히 자세히 들으니 '
 '노방(路傍)의 걸인 악가(樂家)가 타는 그 곡조는 바로 자기 자신의 작곡인 것을 알게 되었습니다. 한편으로 반갑기도 했지만 그러나 저렇게 '
 '함부로 하는 법이 어디 있단 말인가! 그의 난폭한 연주를 듣 고 있던 마스카니는 일시에 격분의 정이 폭발했지만 다시 돌려 생각하니 저 '
 '사람이야 노상에서 구걸하는 서푼 짜리 악가가 아닌가? 구태여 탓할 것도 없 다하고 두 손으로 자기의 귀를 막아버렸습니다. 그러나 부지불 '
 '식중에 그는 다시 그 음악 소리에 귀를 기울이게 되었습니 다. 아무리 참으려 해도 참을 수가 없어서 그는 마침내 방문을 박차고 대문 '
 '간으로 뛰어갔습니다. “이놈아! 아무리 비렁뱅이기로서니 그 따위로 하는 법이 세상에 어디 있 단 말이냐? 그 곡조는 이렇게 하는 '
 '것이다!” 하고 마스카니는 대갈한 후에 걸인의 풍금을 뺏아 들고는 연주법에 대하여 일장 강의를 해주었습니다. 그러나 웬걸, 이번에는 노방 '
 '악가가 대로하여 마스카니에게 시비를 걸며 그의 무례함를 톡톡히 꾸짖었습니다. 그러다가 나중에는 이 사람이 대작곡가 마스카니인 줄을 알게 '
 '되자, 걸인은 머리를 숙여 사과를 하고 돌아갔습니다. 이 사건으로 말미암아 마스카니는 온 종일 앙앙불락하게 지냈습니다.\n')
('label summary :노방의 걸인 악가가 타는 난폭한 곡조가 자신의 곡임을 알게 된 마스카니

아래와 같이 다양한 keyword arguments를 사용하면서 텍스트 생성을 위한 decoding 방법을 수정 가능하다.

In [ ]:
summary = summarizer(
            raw_datasets["test"][0]["document"],
            min_length=MIN_TARGET_LENGTH,
            max_length=MAX_TARGET_LENGTH,
            do_sample=True,
            top_k=50,
            top_p=0.92,
            temperature=2.5,
            num_return_sequences=4,
        )
print('pred summary')
pprint([summ['summary_text'] for summ in summary])

pred summary
['마스카니의 제자 점심때쯤 해서 작곡가 마스카니는 정신을 집중시키기 위하여 책상 앞에 앉 은 후 눈을 감았습니다. 마침 그때 불의에 일어난 '
 '창 밖의 요란한 소리는 그의 환상의 실마리를 끊 어놓았습니다 길거리 창 밑에는 어떤 걸인 풍금수()가 제멋에 겨워 서 한참 신이 '
 '나가지고는 되는 대로 풍금을 타고 있었습니다.',
 '작곡가 마스카니의 제자 점심때쯤 해서 정신을 집중시키기 위하여 책상 앞에 앉 은 후 눈을 감았습니다 . 길거리 창 밑에는 어떤 걸인 '
 '풍금수()가 제멋에 겨워 서 한참 신이 나가지고는 되는 대로 풍금을 타고 있었습니다. 그래서 화가 난 마스카니는 들었던 펜을 내던지고 두 '
 '주먹을 불끈 쥐고 일어나려다가 다 시 가만히 자세히 들으니 노방의 걸인 악가가 타는 그 곡조는 바로 자기 자신의 작곡인 것을 알게 '
 '되었습니다. 그러나 부지불 식중에 그는 다시 그 음악 소리에 귀를 기울이게 되었습니 다. 아무리 비렁뱅이기로서니 그 따',
 '마스카니의 제자 점심때쯤 마스카니는 고음에서 저음으로 아름다운 멜로디가 떠올랐습니다. 마침 그때 불의에 일어난 창 밖의 요란한 소리는 '
 '그의 환상의 실마리를 끊 어놓았습니다. 길거리 창 밑에는 어떤 걸인 풍금수()가 제멋에 겨워 서 한참 신이 나가지고는 되는 대로 풍금을 '
 '타고 있었습니다. 화가 난 마스카니가 두 주먹을 불끈 쥐고 일어나려다가 다 시 가만히 자세히 들으니 노방의 걸인 악가가 타는 그 곡조는 '
 '바로 자기 자신의 작곡인 것을 알게 되었습니다.',
 '마스카니의 제자 점심때쯤 해서 작곡가 마스카니는 정신을 집중시키기 위하여 책상 앞에 앉 은 후 눈을 감았습니다. 저 음에서 고음으로, '
 '고음에서 저음으로 아름다운 멜로디는 그의 머릿속에 떠올랐습니다. 마침 그때 불의에 일어난 창 밖의 요란한 소리는 그의 환상의 실마리를 끊 '
 '어놓았습니다']


Reference :

https://keras.io/examples/nlp/t5_hf_summarization/

https://aifactory.space/learning/detail/2168
